**SIAMESE NETWORK**

**Importing libraries**

In [9]:
import cv2
import tensorflow as tf
import keras
import keras.backend as K

In [6]:
from keras.layers import Conv2D,MaxPool2D,BatchNormalization,Flatten,Dense,Lambda

**The siamese network**

In [3]:
def siamese():
  model=keras.models.Sequential()
  model.add(Conv2D(100,(3,3),activation="relu",kernel_initializer="he_normal",padding="same",input_shape=(128,128,3)))
  model.add((MaxPool2D((2,2),padding="same")))
  model.add(BatchNormalization(axis=-1))
  model.add(Conv2D(500,(3,3),activation="relu",kernel_initializer="he_normal",padding="same"))
  model.add(MaxPool2D((2,2),padding="same"))
  model.add(BatchNormalization(axis=-1))
  model.add(Flatten())
  model.add(Dense(100))
  model.add(Lambda(lambda x: K.l2_normalize(x,axis=-1)))
  return model

In [7]:
model=siamese()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 100)     2800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 100)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 100)       400       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 500)       450500    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 500)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 500)       2000      
_________________________________________________________________
flatten (Flatten)            (None, 512000)           

**Triplet loss layer**

In [11]:
from keras.layers import Layer

In [12]:
class tripletlosslayer(Layer):
  def __init__(self,alpha,**kwargs):
    super(tripletlosslayer,self).__init__(**kwargs)
    self.alpha=alpha
  def tripletloss(self,input):
    Aencod,Pencod,Nencod = input
    dAP=K.sum(K.square(Aencod-Pencod),axis=-1)
    dAN=K.sum(K.square(Aencod-Nencod),axis=-1)
    return (K.maximum(dAP-dAN+self.alpha,0))
  def call(self,input):
    loss= self.tripletloss(input)
    self.add_loss(loss)
    return loss

In [48]:
A_encoding=model(A)
P_encoding=model(P)
N_encoding=model(N)

Net=keras.Model(inputs=[A,P,N],outputs=[out])


ValueError: ignored